In [ ]:
import oracledb

# Replace with your actual oracle connection details
username="sys"
password="password"
dsn="localhost:1521/XEPDB1"

# Connect to oracle
connection = oracledb.connect(
    user=username,
    password=password,
    dsn=dsn,
    mode=oracledb.SYSDBA
)

cursor = connection.cursor()

In [ ]:
cursor.execute("""
CREATE TABLE banks (
    bank_id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    bank_name VARCHAR2(100) UNIQUE NOT NULL
)
""")

cursor.execute("""
CREATE TABLE reviews (
    review_id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    bank_id NUMBER,
    rating NUMBER(2,1),
    review_text CLOB,
    review_date DATE,
    FOREIGN KEY (bank_id) REFERENCES banks(bank_id)
)
""")


In [17]:
# Query data from table
cursor.execute("SELECT * FROM banks")
print(cursor)
rows = cursor.fetchall()

for row in rows:
    print(row)

<oracledb.Cursor on <oracledb.Connection to sys@localhost:1521/XEPDB1>>


In [1]:
import pandas as pd

In [5]:
# Import cleaned CSV
dfAbyssinia = pd.read_csv('../data/abysinia_cleaned_csv.csv')
dfDashen = pd.read_csv('../data/dashen_cleaned_csv.csv')
dfCommercial = pd.read_csv('../data/commercial_cleaned_csv.csv')

dfAllBanks =pd.concat([dfAbyssinia,dfDashen,dfCommercial])

In [12]:
for bank in dfAllBanks['bank_name'].unique():
    cursor.execute("INSERT INTO banks (bank_name) VALUES ('" +bank+"')")

In [13]:
cursor.execute("SELECT * FROM banks")
print(cursor)
rows = cursor.fetchall()

for row in rows:
    print(row)

<oracledb.Cursor on <oracledb.Connection to sys@localhost:1521/XEPDB1>>
(1, 'Bank of Abysinia')
(2, 'Dashen Bank of Ethiopia')
(3, 'Commercial Bank of Ethiopia')


In [19]:
# Declaring bank ids
bank_ids = {
    'Bank of Abysinia':1,
    'Dashen Bank of Ethiopia':2,
    'Commercial Bank of Ethiopia':3
}

In [36]:
# Insert reviews
for _, row in dfAllBanks.iterrows():
    cursor.execute("""
        INSERT INTO reviews (bank_id, rating, review_text, review_date)
        VALUES (:1, :2, :3, TO_DATE(:4, 'YYYY-MM-DD'))
    """, (
        bank_ids[row['bank_name']],
         row['rating'], row['review_text'], row['date']
    ))

In [37]:
cursor.execute("SELECT * FROM reviews")
print(cursor)
rows = cursor.fetchall()

for row in rows:
    print(row)

<oracledb.Cursor on <oracledb.Connection to sys@localhost:1521/XEPDB1>>
(1, 1, 3.0, <oracledb.LOB object at 0x7f6001482c40>, datetime.datetime(2025, 6, 5, 0, 0))
(2, 1, 1.0, <oracledb.LOB object at 0x7f6001482e20>, datetime.datetime(2025, 6, 3, 0, 0))
(3, 1, 5.0, <oracledb.LOB object at 0x7f60014f8f10>, datetime.datetime(2025, 6, 3, 0, 0))
(4, 1, 5.0, <oracledb.LOB object at 0x7f600200f250>, datetime.datetime(2025, 6, 2, 0, 0))
(5, 1, 1.0, <oracledb.LOB object at 0x7f60011b6c40>, datetime.datetime(2025, 6, 1, 0, 0))
(6, 1, 1.0, <oracledb.LOB object at 0x7f600148f9d0>, datetime.datetime(2025, 6, 1, 0, 0))
(7, 1, 5.0, <oracledb.LOB object at 0x7f600148fa90>, datetime.datetime(2025, 6, 1, 0, 0))
(8, 1, 5.0, <oracledb.LOB object at 0x7f60011bc040>, datetime.datetime(2025, 5, 31, 0, 0))
(9, 1, 1.0, <oracledb.LOB object at 0x7f60011bc100>, datetime.datetime(2025, 5, 27, 0, 0))
(10, 1, 3.0, <oracledb.LOB object at 0x7f60011bc190>, datetime.datetime(2025, 5, 25, 0, 0))
(11, 1, 1.0, <oracledb.L